In [1]:
from collections import defaultdict
import copy
import random
import os
import shutil
from urllib.request import urlretrieve
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import plotly.graph_objs as go
import plotly.io as pio


cudnn.benchmark = True

In [2]:
dataset_directory= '/Users/guill/Escritorio/TesisLisboa/archive/DFU'
root_directory = os.path.join(dataset_directory, "Patches")

Abnormal_directory = os.path.join(root_directory, "Abnormal")
Normal_directory = os.path.join(root_directory, "Normal")

In [3]:

Abnormal_images_filepaths = sorted([os.path.join(Abnormal_directory, f) for f in os.listdir(Abnormal_directory)])
Normal_images_filepaths = sorted([os.path.join(Normal_directory, f) for f in os.listdir(Normal_directory)])
images_filepaths = [*Abnormal_images_filepaths, *Normal_images_filepaths]
correct_images_filepaths = [i for i in images_filepaths if cv2.imread(i) is not None]

random.seed(42)
random.shuffle(correct_images_filepaths)

total_images = len(correct_images_filepaths)
train_split = int(total_images * 0.7)
val_split = total_images - train_split

train_images_filepaths = correct_images_filepaths[:train_split]
val_images_filepaths = correct_images_filepaths[train_split:]

print(len(Abnormal_images_filepaths))

print(len(Normal_images_filepaths))

print(len(correct_images_filepaths))

print(len(train_images_filepaths), len(val_images_filepaths))


512
543
1055
738 317


In [4]:
'''
def display_image_grid(images_filepaths, predicted_labels=(), cols=5):
    rows = len(images_filepaths) // cols
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))
    for i, image_filepath in enumerate(images_filepaths):
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        true_label = os.path.normpath(image_filepath).split(os.sep)[-2]
        predicted_label = predicted_labels[i] if predicted_labels else true_label
        color = "green" if true_label == predicted_label else "red"
        ax.ravel()[i].imshow(image)
        ax.ravel()[i].set_title(predicted_label, color=color)
        ax.ravel()[i].set_axis_off()
    plt.tight_layout()
    plt.show()
'''

'\ndef display_image_grid(images_filepaths, predicted_labels=(), cols=5):\n    rows = len(images_filepaths) // cols\n    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))\n    for i, image_filepath in enumerate(images_filepaths):\n        image = cv2.imread(image_filepath)\n        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n        true_label = os.path.normpath(image_filepath).split(os.sep)[-2]\n        predicted_label = predicted_labels[i] if predicted_labels else true_label\n        color = "green" if true_label == predicted_label else "red"\n        ax.ravel()[i].imshow(image)\n        ax.ravel()[i].set_title(predicted_label, color=color)\n        ax.ravel()[i].set_axis_off()\n    plt.tight_layout()\n    plt.show()\n'

In [5]:
'''
for i in range(10):
    img_path = train_images_filepaths[i]
    img = cv2.imread(img_path)
    label = img_path.split('/')[-1].capitalize() # Obtiene la etiqueta según el nombre de archivo
    
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(label)  # Muestra el título de la imagen como la etiqueta correspondiente
    plt.show()
'''

"\nfor i in range(10):\n    img_path = train_images_filepaths[i]\n    img = cv2.imread(img_path)\n    label = img_path.split('/')[-1].capitalize() # Obtiene la etiqueta según el nombre de archivo\n    \n    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))\n    plt.title(label)  # Muestra el título de la imagen como la etiqueta correspondiente\n    plt.show()\n"

In [6]:


class NormalVsAbnormalDataset(Dataset):
    def __init__(self, images_filepaths, transform=None):
        self.images_filepaths = images_filepaths
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if os.path.normpath(image_filepath).split(os.sep)[-2] == "Normal":
            label = 1.0
        else:
            label = 0.0
        if self.transform is not None:
            image = self.transform(image=image)["image"]
            
        return image, label
        


In [7]:
'''
#adding Luv and cmyk to the rgb tensor
import numpy as np
from PIL import Image

class NormalVsAbnormalDataset(Dataset):
    def __init__(self, images_filepaths, transform=None):
        self.images_filepaths = images_filepaths
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image_RGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_cmyk = np.array(Image.fromarray(image).convert('CMYK'))
        image_cmy = image_cmyk[..., :3]  # Seleccionar solo los canales C, M y Y
        image_luv = cv2.cvtColor(image, cv2.COLOR_RGB2Luv)

        if os.path.normpath(image_filepath).split(os.sep)[-2] == "Normal":
            label = 1.0
        else:
            label = 0.0

        if self.transform is not None:
            image_RGB = self.transform(image=image_RGB)["image"]
            image_luv = self.transform(image=image_luv)["image"]
            image_cmy = self.transform(image=image_cmy)["image"]
          # Convert numpy arrays to PyTorch tensors
        image_RGB = torch.from_numpy(image_RGB.transpose(2, 0, 1))
        image_luv = torch.from_numpy(image_luv.transpose(2, 0, 1))
        image_cmy = torch.from_numpy(np.array(image_cmy).transpose(2, 0, 1))
        #print(image_RGB.shape)
        #print(image_luv.shape)
        #print(image_cmy.shape)



        # Concatenate the three representations of color
        image = torch.cat((image_RGB, image_cmy, image_luv), dim=2)
        #print(image.shape)

        return image, label
'''

'\n#adding Luv and cmyk to the rgb tensor\nimport numpy as np\nfrom PIL import Image\n\nclass NormalVsAbnormalDataset(Dataset):\n    def __init__(self, images_filepaths, transform=None):\n        self.images_filepaths = images_filepaths\n        self.transform = transform\n\n    def __len__(self):\n        return len(self.images_filepaths)\n\n    def __getitem__(self, idx):\n        image_filepath = self.images_filepaths[idx]\n        image = cv2.imread(image_filepath)\n        image_RGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n        image_cmyk = np.array(Image.fromarray(image).convert(\'CMYK\'))\n        image_cmy = image_cmyk[..., :3]  # Seleccionar solo los canales C, M y Y\n        image_luv = cv2.cvtColor(image, cv2.COLOR_RGB2Luv)\n\n        if os.path.normpath(image_filepath).split(os.sep)[-2] == "Normal":\n            label = 1.0\n        else:\n            label = 0.0\n\n        if self.transform is not None:\n            image_RGB = self.transform(image=image_RGB)["image"]\

In [8]:
train_transform = A.Compose(
    [
        A.SmallestMaxSize(max_size=160),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=20, p=0.5),#change and try other numbers
        A.RandomCrop(height=128, width=128),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1),brightness_by_max=True, p=0.7),
        A.GaussNoise(var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=False, p=0.5),#meter ruido gausiano
        #A.HueSaturationValue (hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, always_apply=False, p=0.5),#Cambiar aleatoriamente el tono, la saturación y el valor de la imagen de entrada
        A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=10, val_shift_limit=5, always_apply=False, p=0.5),

        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(), #solo lo quito si en la funcion de arriba estoy ya creando el tensor
    ]
)

train_dataset = NormalVsAbnormalDataset(images_filepaths=train_images_filepaths, transform=train_transform)


In [9]:

val_transform = A.Compose(
    [
        A.SmallestMaxSize(max_size=160),
        A.CenterCrop(height=128, width=128),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),  #solo lo quito si en la funcion de arriba estoy ya creando el tensor
    ]
)
val_dataset = NormalVsAbnormalDataset(images_filepaths=val_images_filepaths, transform=val_transform)

In [10]:
'''
#funcion para visualizar las tres versiones de color: RGB LUV y CMYK

def visualize_augmentations(dataset, idx=0, samples=2, cols=1):
    dataset = copy.deepcopy(dataset)
    dataset.transform = A.Compose([t for t in dataset.transform.transforms if not isinstance(t, (A.Normalize, ToTensorV2))])
    rows = samples // cols
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))
    for i in range(samples):
        image, _ = dataset[idx]
        image_rgb = image[:3, :, :]
        image_luv = image[3:6, :, :]
        image_cmy = image[6:, :, :]
        image = torch.cat((image_rgb, image_cmy, image_luv), dim=0)
        image = image.permute(1, 2, 0).numpy()
        ax.ravel()[i].imshow(image)
        ax.ravel()[i].set_axis_off()
    plt.tight_layout()
    plt.show()

'''

'\n#funcion para visualizar las tres versiones de color: RGB LUV y CMYK\n\ndef visualize_augmentations(dataset, idx=0, samples=2, cols=1):\n    dataset = copy.deepcopy(dataset)\n    dataset.transform = A.Compose([t for t in dataset.transform.transforms if not isinstance(t, (A.Normalize, ToTensorV2))])\n    rows = samples // cols\n    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))\n    for i in range(samples):\n        image, _ = dataset[idx]\n        image_rgb = image[:3, :, :]\n        image_luv = image[3:6, :, :]\n        image_cmy = image[6:, :, :]\n        image = torch.cat((image_rgb, image_cmy, image_luv), dim=0)\n        image = image.permute(1, 2, 0).numpy()\n        ax.ravel()[i].imshow(image)\n        ax.ravel()[i].set_axis_off()\n    plt.tight_layout()\n    plt.show()\n\n'

In [11]:
'''
def visualize_augmentations(dataset, idx=0, samples=15, cols=5):
    dataset = copy.deepcopy(dataset)
    dataset.transform = A.Compose([t for t in dataset.transform.transforms if not isinstance(t, (A.Normalize, ToTensorV2))])
    rows = samples // cols
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))
    for i in range(samples):
        image, _ = dataset[idx]
        ax.ravel()[i].imshow(image)
        ax.ravel()[i].set_axis_off()
    plt.tight_layout()
    plt.show()
'''

'\ndef visualize_augmentations(dataset, idx=0, samples=15, cols=5):\n    dataset = copy.deepcopy(dataset)\n    dataset.transform = A.Compose([t for t in dataset.transform.transforms if not isinstance(t, (A.Normalize, ToTensorV2))])\n    rows = samples // cols\n    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))\n    for i in range(samples):\n        image, _ = dataset[idx]\n        ax.ravel()[i].imshow(image)\n        ax.ravel()[i].set_axis_off()\n    plt.tight_layout()\n    plt.show()\n'

In [12]:
'''
random.seed(46)
visualize_augmentations(train_dataset)
'''

'\nrandom.seed(46)\nvisualize_augmentations(train_dataset)\n'

In [13]:
def calculate_accuracy(output, target):
    output = torch.sigmoid(output) >= 0.5
    target = target == 1.0
    return torch.true_divide((target == output).sum(dim=0), output.size(0)).item()


class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]

        metric["val"] += float(val)
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]
    def get_avg(self, metric_name):
        values = self.metrics[metric_name].values()
        avg = sum([value for value in values if isinstance(value, (float, int))]) / len(values)
        return avg
    def __str__(self):
        return " | ".join(
            [
                "{metric_name}: {avg:.{float_precision}f}".format(
                    metric_name=metric_name, avg=metric["avg"], float_precision=self.float_precision
                )
                for (metric_name, metric) in self.metrics.items()
            ]
        )

In [14]:
params = {
    
    #otros modelos:"resnet18""alexnet""vgg16""densenet121" ResNet18, ResNet34, ResNet50, ResNet101, ResNet152
#VGG11, VGG13, VGG16, VGG19
#DenseNet121, DenseNet161, DenseNet169, DenseNet201
#InceptionV3, InceptionResNetV2
#MobileNetV2, MobileNetV3
#EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7
    "model": "efficientNetB2",
    "device": "cpu",  # usar 'cuda' para usar la GPU
    "lr": 0.001, #cambiar a 0.01
    "batch_size": 64,#cambiar a 32
    "num_workers": 0,
    "epochs": 10,
}
#params["model"] = MyModel().to(params["device"])

In [15]:
#Esto funciona siempre
#model = getattr(models, params["model"])(weights=None, num_classes=1,)
#model = model.to(params["device"])
#criterion = nn.BCEWithLogitsLoss().to(params["device"])
#optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])

In [16]:
'''

#ResNet101

from torchvision.models import resnet101, ResNet101_Weights

model = getattr(models, params["model"])(weights=ResNet101_Weights.DEFAULT)  # Cargar ResNet101 con pesos pre-entrenados

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)
model = model.to(params["device"])

criterion = nn.BCEWithLogitsLoss().to(params["device"])
optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])

'''

'\n\n#ResNet101\n\nfrom torchvision.models import resnet101, ResNet101_Weights\n\nmodel = getattr(models, params["model"])(weights=ResNet101_Weights.DEFAULT)  # Cargar ResNet101 con pesos pre-entrenados\n\nnum_ftrs = model.fc.in_features\nmodel.fc = nn.Linear(num_ftrs, 1)\nmodel = model.to(params["device"])\n\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\noptimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])\n\n'

In [17]:
'''

#solo congelo la ultimas 2 capas

import torch.nn as nn
from torchvision.models import resnet101, ResNet101_Weights

model = getattr(models, params["model"])(weights=ResNet101_Weights.DEFAULT)  # Cargar ResNet101 con pesos pre-entrenados

# Congelar todas las capas excepto las últimas
for name, param in model.named_parameters():
    if not any(layer in name for layer in ['layer4', 'fc']):
        param.requires_grad = False


# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])

'''

'\n\n#solo congelo la ultimas 2 capas\n\nimport torch.nn as nn\nfrom torchvision.models import resnet101, ResNet101_Weights\n\nmodel = getattr(models, params["model"])(weights=ResNet101_Weights.DEFAULT)  # Cargar ResNet101 con pesos pre-entrenados\n\n# Congelar todas las capas excepto las últimas\nfor name, param in model.named_parameters():\n    if not any(layer in name for layer in [\'layer4\', \'fc\']):\n        param.requires_grad = False\n\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.fc.in_features\nmodel.fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])\n\n'

In [18]:
'''

#solo congelo la ultimas 5 capas

import torch.nn as nn
from torchvision.models import resnet101, ResNet101_Weights

model = getattr(models, params["model"])(weights=ResNet101_Weights.DEFAULT)  # Cargar ResNet101 con pesos pre-entrenados

# Congelar todas las capas excepto las últimas
for name, param in model.named_parameters():
    if not any(layer in name for layer in ['layer4', 'fc', 'layer3.22', 'layer3.21', 'layer3.20']):
        param.requires_grad = False


# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''


'\n\n#solo congelo la ultimas 5 capas\n\nimport torch.nn as nn\nfrom torchvision.models import resnet101, ResNet101_Weights\n\nmodel = getattr(models, params["model"])(weights=ResNet101_Weights.DEFAULT)  # Cargar ResNet101 con pesos pre-entrenados\n\n# Congelar todas las capas excepto las últimas\nfor name, param in model.named_parameters():\n    if not any(layer in name for layer in [\'layer4\', \'fc\', \'layer3.22\', \'layer3.21\', \'layer3.20\']):\n        param.requires_grad = False\n\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.fc.in_features\nmodel.fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])\n'

In [19]:
'''

#solo congelo la ultimas 7 capas

import torch.nn as nn
from torchvision.models import resnet101, ResNet101_Weights

model = getattr(models, params["model"])(weights=ResNet101_Weights.DEFAULT)  # Cargar ResNet101 con pesos pre-entrenados

# Congelar todas las capas excepto las últimas
for name, param in model.named_parameters():
    if not any(layer in name for layer in ['layer4', 'fc', 'layer3.22', 'layer3.21', 'layer3.20','layer3.19', 'layer3.18', 'layer3.17']):
        param.requires_grad = False


# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''


'\n\n#solo congelo la ultimas 7 capas\n\nimport torch.nn as nn\nfrom torchvision.models import resnet101, ResNet101_Weights\n\nmodel = getattr(models, params["model"])(weights=ResNet101_Weights.DEFAULT)  # Cargar ResNet101 con pesos pre-entrenados\n\n# Congelar todas las capas excepto las últimas\nfor name, param in model.named_parameters():\n    if not any(layer in name for layer in [\'layer4\', \'fc\', \'layer3.22\', \'layer3.21\', \'layer3.20\',\'layer3.19\', \'layer3.18\', \'layer3.17\']):\n        param.requires_grad = False\n\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.fc.in_features\nmodel.fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params[

In [20]:
'''
#resnet50 


from torchvision.models import resnet50, ResNet50_Weights

model = getattr(models, params["model"])(weights=ResNet50_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red


num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)
model = model.to(params["device"])

criterion = nn.BCEWithLogitsLoss().to(params["device"])
optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])
'''

'\n#resnet50 \n\n\nfrom torchvision.models import resnet50, ResNet50_Weights\n\nmodel = getattr(models, params["model"])(weights=ResNet50_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red\n\n\nnum_ftrs = model.fc.in_features\nmodel.fc = nn.Linear(num_ftrs, 1)\nmodel = model.to(params["device"])\n\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\noptimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])\n'

In [21]:
'''
#solo congelo la ultima capa

import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

model = getattr(models, params["model"])(weights=ResNet50_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red

# Congelar todas las capas excepto la última capa lineal
for name, param in model.named_parameters():
    if "fc" not in name:
        param.requires_grad = False

# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''

'\n#solo congelo la ultima capa\n\nimport torch.nn as nn\nfrom torchvision.models import resnet50, ResNet50_Weights\n\nmodel = getattr(models, params["model"])(weights=ResNet50_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red\n\n# Congelar todas las capas excepto la última capa lineal\nfor name, param in model.named_parameters():\n    if "fc" not in name:\n        param.requires_grad = False\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.fc.in_features\nmodel.fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])\n'

In [22]:
'''
#congelar todas las capas menos las 5 ultimas
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

model = getattr(models, params["model"])(weights=ResNet50_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red


# Congelar todas las capas excepto las últimas
for name, param in model.named_parameters():
    if not any(layer in name for layer in ['layer4', 'fc']):
        param.requires_grad = False



# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])

'''

'\n#congelar todas las capas menos las 5 ultimas\nimport torch.nn as nn\nfrom torchvision.models import resnet50, ResNet50_Weights\n\nmodel = getattr(models, params["model"])(weights=ResNet50_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red\n\n\n# Congelar todas las capas excepto las últimas\nfor name, param in model.named_parameters():\n    if not any(layer in name for layer in [\'layer4\', \'fc\']):\n        param.requires_grad = False\n\n\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.fc.in_features\nmodel.fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])\n\n'

In [23]:
'''
#congelar todas las capas menos las 7 ultimas
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

model = getattr(models, params["model"])(weights=ResNet50_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red


# Congelar todas las capas excepto las últimas
for name, param in model.named_parameters():
    if not any(layer in name for layer in ['layer4', 'fc', 'layer3.5']):
        param.requires_grad = False



# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''


'\n#congelar todas las capas menos las 7 ultimas\nimport torch.nn as nn\nfrom torchvision.models import resnet50, ResNet50_Weights\n\nmodel = getattr(models, params["model"])(weights=ResNet50_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red\n\n\n# Congelar todas las capas excepto las últimas\nfor name, param in model.named_parameters():\n    if not any(layer in name for layer in [\'layer4\', \'fc\', \'layer3.5\']):\n        param.requires_grad = False\n\n\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.fc.in_features\nmodel.fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])\n'

In [24]:
'''
#congelar todas las capas menos las 8 ultimas
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

model = getattr(models, params["model"])(weights=ResNet50_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red


# Congelar todas las capas excepto las últimas
for name, param in model.named_parameters():
    if not any(layer in name for layer in ['layer4', 'fc', 'layer3.5','layer3.4']):
        param.requires_grad = False



# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''


'\n#congelar todas las capas menos las 8 ultimas\nimport torch.nn as nn\nfrom torchvision.models import resnet50, ResNet50_Weights\n\nmodel = getattr(models, params["model"])(weights=ResNet50_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red\n\n\n# Congelar todas las capas excepto las últimas\nfor name, param in model.named_parameters():\n    if not any(layer in name for layer in [\'layer4\', \'fc\', \'layer3.5\',\'layer3.4\']):\n        param.requires_grad = False\n\n\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.fc.in_features\nmodel.fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])\n'

In [25]:
'''
#congelar todas las capas menos las 2 ultimas
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

model = getattr(models, params["model"])(weights=ResNet50_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red


# Congelar todas las capas excepto las últimas
for name, param in model.named_parameters():
    if not any(layer in name for layer in ['layer4.1', 'fc']):
        param.requires_grad = False



# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''


'\n#congelar todas las capas menos las 2 ultimas\nimport torch.nn as nn\nfrom torchvision.models import resnet50, ResNet50_Weights\n\nmodel = getattr(models, params["model"])(weights=ResNet50_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red\n\n\n# Congelar todas las capas excepto las últimas\nfor name, param in model.named_parameters():\n    if not any(layer in name for layer in [\'layer4.1\', \'fc\']):\n        param.requires_grad = False\n\n\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.fc.in_features\nmodel.fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])\n'

In [26]:
'''
#densenet121

from torchvision.models import densenet121, DenseNet121_Weights
model = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 1)
model = model.to(params["device"])

criterion = nn.BCEWithLogitsLoss().to(params["device"])
optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])
'''


'\n#densenet121\n\nfrom torchvision.models import densenet121, DenseNet121_Weights\nmodel = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red\nnum_ftrs = model.classifier.in_features\nmodel.classifier = nn.Linear(num_ftrs, 1)\nmodel = model.to(params["device"])\n\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\noptimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])\n'

In [27]:
'''
#Densenet121
#congelo 3 capas
import torch.nn as nn
from torchvision.models import densenet121, DenseNet121_Weights

# Cargar la red DenseNet121 pre-entrenada
model = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red
# Congelar las primeras capas
for name, param in model.named_parameters():
    if "denseblock4" not in name and "norm5" not in name:
        param.requires_grad = False

# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''


'\n#Densenet121\n#congelo 3 capas\nimport torch.nn as nn\nfrom torchvision.models import densenet121, DenseNet121_Weights\n\n# Cargar la red DenseNet121 pre-entrenada\nmodel = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red\n# Congelar las primeras capas\nfor name, param in model.named_parameters():\n    if "denseblock4" not in name and "norm5" not in name:\n        param.requires_grad = False\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.classifier.in_features\nmodel.classifier = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])\n'

In [28]:
'''
#Densenet121
#congelo 2 capas, de la denseblock4.denselayer16 hasta la 13
import torch.nn as nn
from torchvision.models import densenet121, DenseNet121_Weights

# Cargar la red DenseNet121 pre-entrenada
model = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red
# Congelar las primeras capas
for name, param in model.named_parameters():
    if "denseblock4.denselayer16" not in name and "norm5" not in name and "denseblock4.denselayer15" not in name and "denseblock4.denselayer14" not in name and "denseblock4.denselayer13" not in name:
        param.requires_grad = False

# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''


'\n#Densenet121\n#congelo 2 capas, de la denseblock4.denselayer16 hasta la 13\nimport torch.nn as nn\nfrom torchvision.models import densenet121, DenseNet121_Weights\n\n# Cargar la red DenseNet121 pre-entrenada\nmodel = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red\n# Congelar las primeras capas\nfor name, param in model.named_parameters():\n    if "denseblock4.denselayer16" not in name and "norm5" not in name and "denseblock4.denselayer15" not in name and "denseblock4.denselayer14" not in name and "denseblock4.denselayer13" not in name:\n        param.requires_grad = False\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.classifier.in_features\nmodel.classifier = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los pa

In [29]:
'''
#congelo todas las capas menos las ultimas 4 capas
import torch.nn as nn
from torchvision.models import densenet121, DenseNet121_Weights

# Cargar la red DenseNet121 pre-entrenada
model = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red
# Congelar las primeras capas
for name, param in model.named_parameters():
    if "transition3" not in name and "denseblock4" not in name and "norm5" not in name:
        param.requires_grad = False


# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''


'\n#congelo todas las capas menos las ultimas 4 capas\nimport torch.nn as nn\nfrom torchvision.models import densenet121, DenseNet121_Weights\n\n# Cargar la red DenseNet121 pre-entrenada\nmodel = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red\n# Congelar las primeras capas\nfor name, param in model.named_parameters():\n    if "transition3" not in name and "denseblock4" not in name and "norm5" not in name:\n        param.requires_grad = False\n\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.classifier.in_features\nmodel.classifier = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])

In [30]:
'''
#congelo todas las capas menos las ultimas 5 capas
import torch.nn as nn
from torchvision.models import densenet121, DenseNet121_Weights

# Cargar la red DenseNet121 pre-entrenada
model = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red

for name, param in model.named_parameters():
    if not name.startswith('denseblock4') and not name.startswith('transition3') and not name.startswith('norm5') and not name.startswith('denseblock3'):
        param.requires_grad = False

# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])

'''


'\n#congelo todas las capas menos las ultimas 5 capas\nimport torch.nn as nn\nfrom torchvision.models import densenet121, DenseNet121_Weights\n\n# Cargar la red DenseNet121 pre-entrenada\nmodel = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red\n\nfor name, param in model.named_parameters():\n    if not name.startswith(\'denseblock4\') and not name.startswith(\'transition3\') and not name.startswith(\'norm5\') and not name.startswith(\'denseblock3\'):\n        param.requires_grad = False\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.classifier.in_features\nmodel.classifier = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torch.optim.Adam(filter(lambda p: p.requires_

In [31]:
'''
#congelo todas las capas menos las ultimas 6 capas
import torch.nn as nn
from torchvision.models import densenet121, DenseNet121_Weights

# Cargar la red DenseNet121 pre-entrenada
model = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red

for name, param in model.named_parameters():
    if not name.startswith('denseblock4') and not name.startswith('transition3') and not name.startswith('norm5') and not name.startswith('denseblock3') and not name.startswith('transition2'):
        param.requires_grad = False

# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''



'\n#congelo todas las capas menos las ultimas 6 capas\nimport torch.nn as nn\nfrom torchvision.models import densenet121, DenseNet121_Weights\n\n# Cargar la red DenseNet121 pre-entrenada\nmodel = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red\n\nfor name, param in model.named_parameters():\n    if not name.startswith(\'denseblock4\') and not name.startswith(\'transition3\') and not name.startswith(\'norm5\') and not name.startswith(\'denseblock3\') and not name.startswith(\'transition2\'):\n        param.requires_grad = False\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.classifier.in_features\nmodel.classifier = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torc

In [32]:
'''
#congelo todas las capas menos las ultimas 7 capas
import torch.nn as nn
from torchvision.models import densenet121, DenseNet121_Weights

# Cargar la red DenseNet121 pre-entrenada
model = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red

for name, param in model.named_parameters():
    if not name.startswith('denseblock4') and not name.startswith('transition3') and not name.startswith('norm5') and not name.startswith('denseblock3') and not name.startswith('transition2') and not name.startswith('denseblock2'):
        param.requires_grad = False

# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''


'\n#congelo todas las capas menos las ultimas 7 capas\nimport torch.nn as nn\nfrom torchvision.models import densenet121, DenseNet121_Weights\n\n# Cargar la red DenseNet121 pre-entrenada\nmodel = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red\n\nfor name, param in model.named_parameters():\n    if not name.startswith(\'denseblock4\') and not name.startswith(\'transition3\') and not name.startswith(\'norm5\') and not name.startswith(\'denseblock3\') and not name.startswith(\'transition2\') and not name.startswith(\'denseblock2\'):\n        param.requires_grad = False\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.classifier.in_features\nmodel.classifier = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros 

In [33]:
'''
#congelo todas las capas menos las ultimas 8 capas
import torch.nn as nn
from torchvision.models import densenet121, DenseNet121_Weights

# Cargar la red DenseNet121 pre-entrenada
model = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red

for name, param in model.named_parameters():
    if not name.startswith('denseblock4') and not name.startswith('transition3') and not name.startswith('norm5') and not name.startswith('denseblock3') and not name.startswith('transition2') and not name.startswith('denseblock2') and not name.startswith('transition1'):
        param.requires_grad = False

# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])

'''


'\n#congelo todas las capas menos las ultimas 8 capas\nimport torch.nn as nn\nfrom torchvision.models import densenet121, DenseNet121_Weights\n\n# Cargar la red DenseNet121 pre-entrenada\nmodel = getattr(models, params["model"])(weights=DenseNet121_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red\n\nfor name, param in model.named_parameters():\n    if not name.startswith(\'denseblock4\') and not name.startswith(\'transition3\') and not name.startswith(\'norm5\') and not name.startswith(\'denseblock3\') and not name.startswith(\'transition2\') and not name.startswith(\'denseblock2\') and not name.startswith(\'transition1\'):\n        param.requires_grad = False\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model.classifier.in_features\nmodel.classifier = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el o

In [34]:
#importas la el modelo y los pesos que necesites
#recortar alguna capa



#from torchvision.models import vgg16
#model = vgg16(pretrained=False)
#num_ftrs = model.classifier[6].in_features
#model.classifier[6] = nn.Linear(num_ftrs, 1)
#model = model.to(params["device"])

#criterion = nn.BCEWithLogitsLoss().to(params["device"])
#optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])

In [35]:
#from torchvision.models import alexnet, AlexNet_Weights
#recortar alguna capa
#model = getattr(models, params["model"])(weights=AlexNet_Weights.DEFAULT)#para usar los pesos pre-entrenados de la red
#num_ftrs = model.classifier[6].in_features
#model.classifier[6] = nn.Linear(num_ftrs, 1) # reemplazar la última capa completamente conectada por una con una salida

#model = model.to(params["device"])

#criterion = nn.BCEWithLogitsLoss().to(params["device"])
#optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])

In [36]:
'''
#EfficientNetB2
#congelo las ultimas 1 capa

import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# Cargar la red EfficientNet pre-entrenada
model = EfficientNet.from_pretrained('efficientnet-b2')


#congelo
for name, param in model.named_parameters():
    if not name.startswith('_conv_head.') and not name.startswith('_fc.') and not name.startswith('_bn1.') and not name.startswith('_blocks.22'): 
        param.requires_grad = False



# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''

'\n#EfficientNetB2\n#congelo las ultimas 1 capa\n\nimport torch.nn as nn\nfrom efficientnet_pytorch import EfficientNet\n\n# Cargar la red EfficientNet pre-entrenada\nmodel = EfficientNet.from_pretrained(\'efficientnet-b2\')\n\n\n#congelo\nfor name, param in model.named_parameters():\n    if not name.startswith(\'_conv_head.\') and not name.startswith(\'_fc.\') and not name.startswith(\'_bn1.\') and not name.startswith(\'_blocks.22\'): \n        param.requires_grad = False\n\n\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model._fc.in_features\nmodel._fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])\n'

In [37]:
'''
#EfficientNetB2
#congelo las ultimas 3 capa

import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# Cargar la red EfficientNet pre-entrenada
model = EfficientNet.from_pretrained('efficientnet-b2')


# Congelar las primeras capas
for name, param in model.named_parameters():
    if not name.startswith('_conv_head.') and not name.startswith('_fc.') and not name.startswith('_bn1.') and not name.startswith('_blocks.22') and not name.startswith('_blocks.21') and not name.startswith('_blocks.20'): 
        param.requires_grad = False

# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''

'\n#EfficientNetB2\n#congelo las ultimas 3 capa\n\nimport torch.nn as nn\nfrom efficientnet_pytorch import EfficientNet\n\n# Cargar la red EfficientNet pre-entrenada\nmodel = EfficientNet.from_pretrained(\'efficientnet-b2\')\n\n\n# Congelar las primeras capas\nfor name, param in model.named_parameters():\n    if not name.startswith(\'_conv_head.\') and not name.startswith(\'_fc.\') and not name.startswith(\'_bn1.\') and not name.startswith(\'_blocks.22\') and not name.startswith(\'_blocks.21\') and not name.startswith(\'_blocks.20\'): \n        param.requires_grad = False\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model._fc.in_features\nmodel._fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torch.optim.Adam(filter(lambda p: p.req

In [38]:
'''
#EfficientNetB2
#congelo las ultimas 4 capa

import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# Cargar la red EfficientNet pre-entrenada
model = EfficientNet.from_pretrained('efficientnet-b2')


# Congelar las primeras capas
for name, param in model.named_parameters():
    if not name.startswith('_conv_head.') and not name.startswith('_fc.') and not name.startswith('_bn1.') and not name.startswith('_blocks.22') and not name.startswith('_blocks.21') and not name.startswith('_blocks.20') and not name.startswith('_blocks.19'): 
        param.requires_grad = False

# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])

'''

'\n#EfficientNetB2\n#congelo las ultimas 4 capa\n\nimport torch.nn as nn\nfrom efficientnet_pytorch import EfficientNet\n\n# Cargar la red EfficientNet pre-entrenada\nmodel = EfficientNet.from_pretrained(\'efficientnet-b2\')\n\n\n# Congelar las primeras capas\nfor name, param in model.named_parameters():\n    if not name.startswith(\'_conv_head.\') and not name.startswith(\'_fc.\') and not name.startswith(\'_bn1.\') and not name.startswith(\'_blocks.22\') and not name.startswith(\'_blocks.21\') and not name.startswith(\'_blocks.20\') and not name.startswith(\'_blocks.19\'): \n        param.requires_grad = False\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model._fc.in_features\nmodel._fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer =

In [39]:
'''
#EfficientNetB2
#congelo las ultimas 5 capa

import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# Cargar la red EfficientNet pre-entrenada
model = EfficientNet.from_pretrained('efficientnet-b2')


# Congelar las primeras capas
for name, param in model.named_parameters():
    if not name.startswith('_conv_head.') and not name.startswith('_fc.') and not name.startswith('_bn1.') and not name.startswith('_blocks.22') and not name.startswith('_blocks.21') and not name.startswith('_blocks.20') and not name.startswith('_blocks.19') and not name.startswith('_blocks.18'): 
        param.requires_grad = False

# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''

'\n#EfficientNetB2\n#congelo las ultimas 5 capa\n\nimport torch.nn as nn\nfrom efficientnet_pytorch import EfficientNet\n\n# Cargar la red EfficientNet pre-entrenada\nmodel = EfficientNet.from_pretrained(\'efficientnet-b2\')\n\n\n# Congelar las primeras capas\nfor name, param in model.named_parameters():\n    if not name.startswith(\'_conv_head.\') and not name.startswith(\'_fc.\') and not name.startswith(\'_bn1.\') and not name.startswith(\'_blocks.22\') and not name.startswith(\'_blocks.21\') and not name.startswith(\'_blocks.20\') and not name.startswith(\'_blocks.19\') and not name.startswith(\'_blocks.18\'): \n        param.requires_grad = False\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model._fc.in_features\nmodel._fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámet

In [40]:
'''
#EfficientNetB2
#congelo las ultimas 6 capa

import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# Cargar la red EfficientNet pre-entrenada
model = EfficientNet.from_pretrained('efficientnet-b2')



# Congelar las primeras capas
for name, param in model.named_parameters():
    if not name.startswith('_conv_head.') and not name.startswith('_fc.') and not name.startswith('_bn1.') and not name.startswith('_blocks.22') and not name.startswith('_blocks.21') and not name.startswith('_blocks.20') and not name.startswith('_blocks.19') and not name.startswith('_blocks.18') and not name.startswith('_blocks.17'): 
        param.requires_grad = False


# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''

'\n#EfficientNetB2\n#congelo las ultimas 6 capa\n\nimport torch.nn as nn\nfrom efficientnet_pytorch import EfficientNet\n\n# Cargar la red EfficientNet pre-entrenada\nmodel = EfficientNet.from_pretrained(\'efficientnet-b2\')\n\n\n\n# Congelar las primeras capas\nfor name, param in model.named_parameters():\n    if not name.startswith(\'_conv_head.\') and not name.startswith(\'_fc.\') and not name.startswith(\'_bn1.\') and not name.startswith(\'_blocks.22\') and not name.startswith(\'_blocks.21\') and not name.startswith(\'_blocks.20\') and not name.startswith(\'_blocks.19\') and not name.startswith(\'_blocks.18\') and not name.startswith(\'_blocks.17\'): \n        param.requires_grad = False\n\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model._fc.in_features\nmodel._fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Defin

In [41]:
'''
#EfficientNetB2
#congelo las ultimas 7 capa

import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# Cargar la red EfficientNet pre-entrenada
model = EfficientNet.from_pretrained('efficientnet-b2')



# Congelar las primeras capas
for name, param in model.named_parameters():
    if not name.startswith('_conv_head.') and not name.startswith('_fc.') and not name.startswith('_bn1.') and not name.startswith('_blocks.22') and not name.startswith('_blocks.21') and not name.startswith('_blocks.20') and not name.startswith('_blocks.19') and not name.startswith('_blocks.18') and not name.startswith('_blocks.17') and not name.startswith('_blocks.16'): 
        param.requires_grad = False


# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''

'\n#EfficientNetB2\n#congelo las ultimas 7 capa\n\nimport torch.nn as nn\nfrom efficientnet_pytorch import EfficientNet\n\n# Cargar la red EfficientNet pre-entrenada\nmodel = EfficientNet.from_pretrained(\'efficientnet-b2\')\n\n\n\n# Congelar las primeras capas\nfor name, param in model.named_parameters():\n    if not name.startswith(\'_conv_head.\') and not name.startswith(\'_fc.\') and not name.startswith(\'_bn1.\') and not name.startswith(\'_blocks.22\') and not name.startswith(\'_blocks.21\') and not name.startswith(\'_blocks.20\') and not name.startswith(\'_blocks.19\') and not name.startswith(\'_blocks.18\') and not name.startswith(\'_blocks.17\') and not name.startswith(\'_blocks.16\'): \n        param.requires_grad = False\n\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model._fc.in_features\nmodel._fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogi

In [42]:

#EfficientNetB2
#congelo las ultimas 8 capa

import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# Cargar la red EfficientNet pre-entrenada
model = EfficientNet.from_pretrained('efficientnet-b2')



# Congelar las primeras capas
for name, param in model.named_parameters():
    if not name.startswith('_conv_head.') and not name.startswith('_fc.') and not name.startswith('_bn1.') and not name.startswith('_blocks.22') and not name.startswith('_blocks.21') and not name.startswith('_blocks.20') and not name.startswith('_blocks.19') and not name.startswith('_blocks.18') and not name.startswith('_blocks.17') and not name.startswith('_blocks.16') and not name.startswith('_blocks.15'): 
        param.requires_grad = False


# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])


Loaded pretrained weights for efficientnet-b2


In [43]:
'''
#EfficientNetB2
#congelo las ultimas 8 capa tuneadas

import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# Cargar la red EfficientNet pre-entrenada
model = EfficientNet.from_pretrained('efficientnet-b2')



# Congelar las primeras capas
for name, param in model.named_parameters():
    if not name.startswith('_conv_head.') and not name.startswith('_fc.') and not name.startswith('_bn1.') and not name.startswith('_blocks.22') and not name.startswith('_blocks.21') and not name.startswith('_blocks.20') and not name.startswith('_blocks.19') and not name.startswith('_blocks.18') and not name.startswith('_blocks.17') and not name.startswith('_blocks.16') and not name.startswith('_blocks.15._bn0') and not name.startswith('_blocks.15._bn1'): 
        param.requires_grad = False


# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''

'\n#EfficientNetB2\n#congelo las ultimas 8 capa tuneadas\n\nimport torch.nn as nn\nfrom efficientnet_pytorch import EfficientNet\n\n# Cargar la red EfficientNet pre-entrenada\nmodel = EfficientNet.from_pretrained(\'efficientnet-b2\')\n\n\n\n# Congelar las primeras capas\nfor name, param in model.named_parameters():\n    if not name.startswith(\'_conv_head.\') and not name.startswith(\'_fc.\') and not name.startswith(\'_bn1.\') and not name.startswith(\'_blocks.22\') and not name.startswith(\'_blocks.21\') and not name.startswith(\'_blocks.20\') and not name.startswith(\'_blocks.19\') and not name.startswith(\'_blocks.18\') and not name.startswith(\'_blocks.17\') and not name.startswith(\'_blocks.16\') and not name.startswith(\'_blocks.15._bn0\') and not name.startswith(\'_blocks.15._bn1\'): \n        param.requires_grad = False\n\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model._fc.in_features\nmodel._fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el c

In [44]:
'''
#EfficientNetB2
#congelo las ultimas 9 capa

import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# Cargar la red EfficientNet pre-entrenada
model = EfficientNet.from_pretrained('efficientnet-b2')


# Congelar las primeras capas
for name, param in model.named_parameters():
    if not name.startswith('_conv_head.') and not name.startswith('_fc.') and not name.startswith('_bn1.') and not name.startswith('_blocks.22') and not name.startswith('_blocks.21') and not name.startswith('_blocks.20') and not name.startswith('_blocks.19') and not name.startswith('_blocks.18') and not name.startswith('_blocks.17') and not name.startswith('_blocks.16') and not name.startswith('_blocks.15') and not name.startswith('_blocks.14'): 
        param.requires_grad = False

# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''

'\n#EfficientNetB2\n#congelo las ultimas 9 capa\n\nimport torch.nn as nn\nfrom efficientnet_pytorch import EfficientNet\n\n# Cargar la red EfficientNet pre-entrenada\nmodel = EfficientNet.from_pretrained(\'efficientnet-b2\')\n\n\n# Congelar las primeras capas\nfor name, param in model.named_parameters():\n    if not name.startswith(\'_conv_head.\') and not name.startswith(\'_fc.\') and not name.startswith(\'_bn1.\') and not name.startswith(\'_blocks.22\') and not name.startswith(\'_blocks.21\') and not name.startswith(\'_blocks.20\') and not name.startswith(\'_blocks.19\') and not name.startswith(\'_blocks.18\') and not name.startswith(\'_blocks.17\') and not name.startswith(\'_blocks.16\') and not name.startswith(\'_blocks.15\') and not name.startswith(\'_blocks.14\'): \n        param.requires_grad = False\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model._fc.in_features\nmodel._fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo 

In [45]:
'''
#EfficientNetB2
#congelo las ultimas 10 capa

import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# Cargar la red EfficientNet pre-entrenada
model = EfficientNet.from_pretrained('efficientnet-b2')


# Congelar las primeras capas
for name, param in model.named_parameters():
    if not name.startswith('_conv_head.') and not name.startswith('_fc.') and not name.startswith('_bn1.') and not name.startswith('_blocks.22') and not name.startswith('_blocks.21') and not name.startswith('_blocks.20') and not name.startswith('_blocks.19') and not name.startswith('_blocks.18') and not name.startswith('_blocks.17') and not name.startswith('_blocks.16') and not name.startswith('_blocks.15') and not name.startswith('_blocks.14') and not name.startswith('_blocks.13'): 
        param.requires_grad = False

# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''

'\n#EfficientNetB2\n#congelo las ultimas 10 capa\n\nimport torch.nn as nn\nfrom efficientnet_pytorch import EfficientNet\n\n# Cargar la red EfficientNet pre-entrenada\nmodel = EfficientNet.from_pretrained(\'efficientnet-b2\')\n\n\n# Congelar las primeras capas\nfor name, param in model.named_parameters():\n    if not name.startswith(\'_conv_head.\') and not name.startswith(\'_fc.\') and not name.startswith(\'_bn1.\') and not name.startswith(\'_blocks.22\') and not name.startswith(\'_blocks.21\') and not name.startswith(\'_blocks.20\') and not name.startswith(\'_blocks.19\') and not name.startswith(\'_blocks.18\') and not name.startswith(\'_blocks.17\') and not name.startswith(\'_blocks.16\') and not name.startswith(\'_blocks.15\') and not name.startswith(\'_blocks.14\') and not name.startswith(\'_blocks.13\'): \n        param.requires_grad = False\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model._fc.in_features\nmodel._fc = nn.Linear(num_ftrs, 1)\n\n# Move

In [46]:
'''

from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b2')
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)
model = model.to(params["device"])

criterion = nn.BCEWithLogitsLoss().to(params["device"])
optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])

'''

'\n\nfrom efficientnet_pytorch import EfficientNet\n\nmodel = EfficientNet.from_pretrained(\'efficientnet-b2\')\nnum_ftrs = model._fc.in_features\nmodel._fc = nn.Linear(num_ftrs, 1)\nmodel = model.to(params["device"])\n\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\noptimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])\n\n'

In [47]:
'''

from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b3')
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)
model = model.to(params["device"])

criterion = nn.BCEWithLogitsLoss().to(params["device"])
optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])

'''

'\n\nfrom efficientnet_pytorch import EfficientNet\n\nmodel = EfficientNet.from_pretrained(\'efficientnet-b3\')\nnum_ftrs = model._fc.in_features\nmodel._fc = nn.Linear(num_ftrs, 1)\nmodel = model.to(params["device"])\n\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\noptimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])\n\n'

In [48]:
'''

#efficientNetB3
#congelo todas menos las ultimas 2 capa
from efficientnet_pytorch import EfficientNet
import torch.nn as nn

# Cargar la red EfficientNet-B3 pre-entrenada
model = EfficientNet.from_pretrained('efficientnet-b3')

#congelo
for name, param in model.named_parameters():
    if not name.startswith('_bn1.') and not name.startswith('_fc.') and not name.startswith('_conv_head.') and not name.startswith('_blocks.25') and not name.startswith('_blocks.24'):
        param.requires_grad = False

# Reemplazar el clasificador lineal con uno personalizado
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)

# Mover el modelo y el criterio al dispositivo especificado
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])

# Definir el optimizador y ajustar solo los parámetros que no están congelados
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])
'''

'\n\n#efficientNetB3\n#congelo todas menos las ultimas 2 capa\nfrom efficientnet_pytorch import EfficientNet\nimport torch.nn as nn\n\n# Cargar la red EfficientNet-B3 pre-entrenada\nmodel = EfficientNet.from_pretrained(\'efficientnet-b3\')\n\n#congelo\nfor name, param in model.named_parameters():\n    if not name.startswith(\'_bn1.\') and not name.startswith(\'_fc.\') and not name.startswith(\'_conv_head.\') and not name.startswith(\'_blocks.25\') and not name.startswith(\'_blocks.24\'):\n        param.requires_grad = False\n\n# Reemplazar el clasificador lineal con uno personalizado\nnum_ftrs = model._fc.in_features\nmodel._fc = nn.Linear(num_ftrs, 1)\n\n# Mover el modelo y el criterio al dispositivo especificado\nmodel = model.to(params["device"])\ncriterion = nn.BCEWithLogitsLoss().to(params["device"])\n\n# Definir el optimizador y ajustar solo los parámetros que no están congelados\noptimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=params["lr"])

In [49]:
train_loader = DataLoader(
    train_dataset, batch_size=params["batch_size"], shuffle=True, num_workers=params["num_workers"], pin_memory=True,
)
val_loader = DataLoader(
    val_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params["num_workers"], pin_memory=True,
)

In [50]:
'''
def train(train_loader, model, criterion, optimizer, epoch, params):
    metric_monitor = MetricMonitor()
    model.train()
    stream = tqdm(train_loader)
    for i, (images, target) in enumerate(stream, start=1):
        images = images.to(params["device"], non_blocking=True)
        target = target.to(params["device"], non_blocking=True).float().view(-1, 1)
        
        output = model(images)
        loss = criterion(output, target)
        accuracy = calculate_accuracy(output, target)
        metric_monitor.update("Loss", loss.item())
        metric_monitor.update("Accuracy", accuracy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        stream.set_description(
            "Epoch: {epoch}. Train.      {metric_monitor}".format(epoch=epoch, metric_monitor=metric_monitor)
        )
'''    

'\ndef train(train_loader, model, criterion, optimizer, epoch, params):\n    metric_monitor = MetricMonitor()\n    model.train()\n    stream = tqdm(train_loader)\n    for i, (images, target) in enumerate(stream, start=1):\n        images = images.to(params["device"], non_blocking=True)\n        target = target.to(params["device"], non_blocking=True).float().view(-1, 1)\n        \n        output = model(images)\n        loss = criterion(output, target)\n        accuracy = calculate_accuracy(output, target)\n        metric_monitor.update("Loss", loss.item())\n        metric_monitor.update("Accuracy", accuracy)\n        optimizer.zero_grad()\n        loss.backward()\n        optimizer.step()\n        stream.set_description(\n            "Epoch: {epoch}. Train.      {metric_monitor}".format(epoch=epoch, metric_monitor=metric_monitor)\n        )\n'

In [51]:
'''
def validate(val_loader, model, criterion, epoch, params):
    metric_monitor = MetricMonitor()
    model.eval()
    stream = tqdm(val_loader)
    with torch.no_grad():
        for i, (images, target) in enumerate(stream, start=1):
            images = images.to(params["device"], non_blocking=True)
            target = target.to(params["device"], non_blocking=True).float().view(-1, 1)
            
            output = model(images)
            loss = criterion(output, target)
            accuracy = calculate_accuracy(output, target)

            metric_monitor.update("Loss", loss.item())
            metric_monitor.update("Accuracy", accuracy)
            stream.set_description(
                "Epoch: {epoch}. Validation. {metric_monitor}".format(epoch=epoch, metric_monitor=metric_monitor)
            )
    return metric_monitor.get_avg("Loss")
    
'''

'\ndef validate(val_loader, model, criterion, epoch, params):\n    metric_monitor = MetricMonitor()\n    model.eval()\n    stream = tqdm(val_loader)\n    with torch.no_grad():\n        for i, (images, target) in enumerate(stream, start=1):\n            images = images.to(params["device"], non_blocking=True)\n            target = target.to(params["device"], non_blocking=True).float().view(-1, 1)\n            \n            output = model(images)\n            loss = criterion(output, target)\n            accuracy = calculate_accuracy(output, target)\n\n            metric_monitor.update("Loss", loss.item())\n            metric_monitor.update("Accuracy", accuracy)\n            stream.set_description(\n                "Epoch: {epoch}. Validation. {metric_monitor}".format(epoch=epoch, metric_monitor=metric_monitor)\n            )\n    return metric_monitor.get_avg("Loss")\n    \n'

In [52]:
def apply_spatial_attention(images, model):
    in_channels = model._blocks[1]._depthwise_conv.out_channels

    spatial_attention = SpatialAttention(in_channels=3)


    
    images_with_attention, attention = spatial_attention(images)  # Aplicar el Spatial Attention a las imágenes
    output = model(images_with_attention)  # Pasar las imágenes con atención por el modelo
    
    return output, attention


In [53]:
import torch
import torch.nn as nn

class SpatialAttention(nn.Module):
    def __init__(self, in_channels):
        super(SpatialAttention, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, 1, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Calcular los pesos de atención
        weights = self.conv(x)
        weights = self.sigmoid(weights)
        
        # Aplicar los pesos de atención a la entrada
        x = x * weights
        
        return x, weights


In [54]:
def train(train_loader, model, criterion, optimizer, epoch, params):
    metric_monitor = MetricMonitor()
    model.train()
    stream = tqdm(train_loader)
    
    for i, (images, target) in enumerate(stream, start=1):
        images = images.to(params["device"], non_blocking=True)
        target = target.to(params["device"], non_blocking=True).float().view(-1, 1)
        
        output, attention = apply_spatial_attention(images, model)

        loss = criterion(output, target)
        accuracy = calculate_accuracy(output, target)
        
        metric_monitor.update("Loss", loss.item())
        metric_monitor.update("Accuracy", accuracy)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        stream.set_description(
            "Epoch: {epoch}. Train. {metric_monitor}".format(epoch=epoch, metric_monitor=metric_monitor)
        )

def validate(val_loader, model, criterion, epoch, params):
    metric_monitor = MetricMonitor()
    model.eval()
    stream = tqdm(val_loader)
    
    with torch.no_grad():
        for i, (images, target) in enumerate(stream, start=1):
            images = images.to(params["device"], non_blocking=True)
            target = target.to(params["device"], non_blocking=True).float().view(-1, 1)
            
            output, attention = apply_spatial_attention(images, model)

            loss = criterion(output, target)
            accuracy = calculate_accuracy(output, target)

            metric_monitor.update("Loss", loss.item())
            metric_monitor.update("Accuracy", accuracy)
            
            stream.set_description(
                "Epoch: {epoch}. Validation. {metric_monitor}".format(epoch=epoch, metric_monitor=metric_monitor)
            )
    
    return metric_monitor.get_avg("Loss")


In [55]:
# inicializa listas para guardar la pérdida y la precisión durante el entrenamiento
best_loss = float("inf")  # inicializa el mejor valor de pérdida a infinito
best_model_path = None  # inicializa el mejor modelo como vacío

for epoch in range(1, params["epochs"] + 1):
    train(train_loader, model, criterion, optimizer, epoch, params)
    val_loss = validate(val_loader, model, criterion, epoch, params)

    if val_loss < best_loss:
        best_loss = val_loss
        best_model_path = "best_model_EfficientNetB2_spatialattention.pth"
        torch.save(model.state_dict(), best_model_path)
        
print("Best model saved at", best_model_path)

Epoch: 1. Train. Loss: 0.278 | Accuracy: 0.897: 100%|██████████████████████████████████| 12/12 [02:48<00:00, 14.06s/it]
Epoch: 1. Validation. Loss: 0.097 | Accuracy: 0.965: 100%|███████████████████████████████| 5/5 [00:25<00:00,  5.13s/it]
Epoch: 2. Train. Loss: 0.076 | Accuracy: 0.975: 100%|██████████████████████████████████| 12/12 [02:48<00:00, 14.08s/it]
Epoch: 2. Validation. Loss: 0.222 | Accuracy: 0.949: 100%|███████████████████████████████| 5/5 [00:25<00:00,  5.08s/it]
Epoch: 3. Train. Loss: 0.084 | Accuracy: 0.973: 100%|██████████████████████████████████| 12/12 [02:49<00:00, 14.10s/it]
Epoch: 3. Validation. Loss: 0.075 | Accuracy: 0.981: 100%|███████████████████████████████| 5/5 [00:25<00:00,  5.19s/it]
Epoch: 4. Train. Loss: 0.066 | Accuracy: 0.979: 100%|██████████████████████████████████| 12/12 [02:51<00:00, 14.27s/it]
Epoch: 4. Validation. Loss: 0.094 | Accuracy: 0.988: 100%|███████████████████████████████| 5/5 [00:27<00:00,  5.46s/it]
Epoch: 5. Train. Loss: 0.073 | Accuracy:

KeyboardInterrupt: 

In [ ]:
'''

import matplotlib.pyplot as plt

# Define the data
epochs = range(1, 11)
train_losses = [0.238, 0.084, 0.082, 0.034, 0.074, 0.036, 0.042, 0.014, 0.086, 0.055]
val_losses = [0.146, 0.189, 0.291, 0.050, 0.063, 0.044, 0.060, 0.036, 0.000, 0.006]
train_accs = [0.899, 0.980, 0.977, 0.991, 0.977, 0.990, 0.993, 0.996, 0.975, 0.978]
val_accs = [0.971, 0.978, 0.968, 0.990, 0.994, 0.994, 0.994, 0.990, 1.000, 0.997]

#train_losses 
#val_losses 
#train_accs 
#val_accs 



# Create the subplots
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(8, 8))

# Plot the training and validation losses
#ax1.set_ylim(bottom=0)
ax1.plot(epochs, train_losses, label='train loss')
ax1.plot(epochs, val_losses, label='val loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()

# Plot the training and validation accuracies
ax2.set_ylim(bottom=0)
ax2.plot(epochs, train_accs, label='train acc')
ax2.plot(epochs, val_accs, label='val acc')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()

# Guardar el plot con un nombre personalizado
#plt.savefig('lr=0,01_batch_size=64_model=EfficientNetB5_frozenLast10.pdf')

# Show the plot
plt.show()

'''

In [ ]:
#Load the model
import torch

# = YourModel()
model.load_state_dict(torch.load("best_model_EfficientNetB2_spatialattention.pth"))


In [ ]:

class NormalVsAbnormalInferenceDataset(Dataset):
    def __init__(self, images_filepaths, labels, transform=None):
        self.images_filepaths = images_filepaths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        
        label = self.labels[idx]
        
        return image, label


In [ ]:
'''
class NormalVsAbnormalInferenceDataset(Dataset):
    def __init__(self, images_filepaths,labels, transform=None):
        self.images_filepaths = images_filepaths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        

        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image_RGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_cmyk = np.array(Image.fromarray(image).convert('CMYK'))
        image_cmy = image_cmyk[..., :3]  # Seleccionar solo los canales C, M y Y
        image_luv = cv2.cvtColor(image, cv2.COLOR_RGB2Luv)

        
        if self.transform is not None:
            image_RGB = self.transform(image=image_RGB)["image"]
            image_luv = self.transform(image=image_luv)["image"]
            image_cmy = self.transform(image=image_cmy)["image"]
          # Convert numpy arrays to PyTorch tensors
        image_RGB = torch.from_numpy(image_RGB.transpose(2, 0, 1))
        image_luv = torch.from_numpy(image_luv.transpose(2, 0, 1))
        image_cmy = torch.from_numpy(np.array(image_cmy).transpose(2, 0, 1))
        #print(image_RGB.shape)
        #print(image_luv.shape)
        #print(image_cmy.shape)


        label = self.labels[idx]
        # Concatenate the three representations of color
        image = torch.cat((image_RGB, image_cmy, image_luv), dim=2)
        #print(image.shape)

        return image, label
        
'''

In [ ]:
test_transform = A.Compose(
    [
        A.SmallestMaxSize(max_size=160),
        A.CenterCrop(height=128, width=128),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(), #solo lo quito si en la funcion de arriba estoy ya creando el tensor
        
    ]
)


In [ ]:

import re

def get_test_labels(images_filepaths):
    labels = []
    for filepath in images_filepaths:
        match = re.search(r"\d+", filepath)
        if match and int(match.group()) <= 143:
            labels.append(0)
        else:
            labels.append(1)
    return labels


In [ ]:
'''
def get_test_labels(images_filepaths):
    labels = []
    for filepath in images_filepaths:
        if "Normal" in filepath:
            labels.append(1)
        else:
            labels.append(0)
    return labels
'''

In [ ]:
test_image_folder = '/Users/guill/Escritorio/TesisLisboa/archive/DFU/TestSet/test'

test_image_filepaths = [os.path.join(test_image_folder, f) for f in os.listdir(test_image_folder) if f.endswith('.jpg') or f.endswith('.png') ]

#print(test_image_filepaths)

test_image_labels = get_test_labels(test_image_filepaths) # lista de etiquetas, en el mismo orden que las imágenes de prueba
#print(test_image_labels)

test_dataset = NormalVsAbnormalInferenceDataset(test_image_filepaths, test_image_labels, transform=test_transform)
#print(test_dataset)
test_dataloader = DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params["num_workers"], pin_memory=True)
#print(test_dataloader)

In [ ]:
'''
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns

model = model.eval()
predicted_labels = []
true_labels = []

with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(params["device"], non_blocking=True)
        labels = labels.to(params["device"], non_blocking=True)
        output = model(images)
        predictions = torch.sigmoid(output).cpu().numpy()
        predicted_labels += predictions.tolist()
        true_labels += labels.cpu().numpy().tolist()

predicted_labels = np.array(predicted_labels)  # Convert to numpy array


# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, [1 if pred > 0.5 else 0 for pred in predicted_labels])

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='d')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix EfficientNetB2')
plt.savefig('confusion_matrix_efficientNetB2.png') 
plt.show()
'''


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt


model = model.eval()
predicted_labels = []
with torch.no_grad():
    for images,labels in test_dataloader:
        images = images.to(params["device"], non_blocking=True)
        labels = labels.to(params["device"], non_blocking=True)
        output = model(images)
        predictions = torch.sigmoid(output).cpu().numpy()
        predicted_labels += predictions.tolist()
        

fpr, tpr, thresholds = roc_curve(test_image_labels, predicted_labels, drop_intermediate=False)
roc_auc = auc(fpr, tpr)


plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")

plt.savefig('roc_curve_EfficientNetB2_spatialattention.png') # Guarda la imagen como 'roc_curve.png'
plt.show()

In [ ]:
'''
print(len(test_image_labels))
print(len(predicted_labels))
print(test_image_labels)
print(predicted_labels)
'''

In [ ]:

#F1-Score Normalizado

from sklearn.metrics import f1_score
import numpy as np
import pandas as pd

model = model.eval()
predicted_labels = []
with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(params["device"], non_blocking=True)
        labels = labels.to(params["device"], non_blocking=True)
        output = model(images)
        predictions = torch.sigmoid(output).cpu().numpy()
        predicted_labels += predictions.tolist()


predicted_labels_discrete = np.where(np.array(predicted_labels) >= 0.5, 1, 0)


# Calcular el F1-score normalizado
f1score = f1_score(test_image_labels, predicted_labels_discrete, average='weighted')

print("F1-score normalizado:", f1score)


# Verificar si existe el archivo de resultados
if os.path.isfile('resultados_F1Score.xlsx'):
    # Cargar el archivo existente en un DataFrame de pandas
    results_df = pd.read_excel('resultados_F1Score.xlsx')
else:
    # Si no existe, crear un DataFrame vacío
    results_df = pd.DataFrame(columns=['Modelo', 'F1-score'])

# Agregar los resultados del nuevo modelo al DataFrame
new_row = pd.DataFrame({'Modelo': ['EfficientNetB2_spatialattention'], 'F1-score': [f1score]})
results_df = pd.concat([results_df, new_row], ignore_index=True)

# Guardar el DataFrame en un archivo de Excel
results_df.to_excel('resultados_F1Score.xlsx', index=False)




In [ ]:


from sklearn.metrics import average_precision_score
import numpy as np
import pandas as pd

model = model.eval()
predicted_labels = []
with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(params["device"], non_blocking=True)
        labels = labels.to(params["device"], non_blocking=True)
        output = model(images)
        predictions = torch.sigmoid(output).cpu().numpy()
        predicted_labels += predictions.tolist()

predicted_labels_discrete = np.where(np.array(predicted_labels) >= 0.5, 1, 0)

# Calcular el MAP
map_score = average_precision_score(test_image_labels, predicted_labels_discrete, average='macro')

print("MAP:", map_score)


import os.path

# Verificar si existe el archivo de resultados
if os.path.isfile('resultados_MAP.xlsx'):
    # Cargar el archivo existente en un DataFrame de pandas
    resultados = pd.read_excel('resultados_MAP.xlsx')
else:
    # Si no existe, crear un DataFrame vacío
    resultados = pd.DataFrame(columns=['Modelo', 'MAP'])

# Agregar los resultados del nuevo modelo al DataFrame
new_row = pd.DataFrame({'Modelo': ['EfficientNetB2_spatialattention'], 'MAP': [map_score]})
resultados = pd.concat([resultados, new_row], ignore_index=True)

# Guardar el DataFrame en un archivo de Excel
resultados.to_excel('resultados_MAP.xlsx', index=False)


In [ ]:
'''
#Evaluacion del test con sus respectivos porcentajes
import cv2

model = model.eval()
predicted_probs = []
predicted_labels = []
with torch.no_grad():
    for images,labels in test_dataloader:
        images = images.to(params["device"], non_blocking=True)
        output = model(images)
        probabilities = torch.sigmoid(output).cpu().numpy()
        predictions = (torch.sigmoid(output) >= 0.5)[:, 0]
        predicted_labels += ["Normal" if is_normal else "Abnormal" for is_normal in predictions]
        predicted_probs += [probs.tolist() for probs in probabilities]


# Imprimir el nombre de la imagen y su etiqueta
for i, filename in enumerate(os.listdir(test_image_folder)):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        if i < len(predicted_labels):
            label = predicted_labels[i]
            prob_normal = predicted_probs[i][0]
            prob_abnormal = 1 - prob_normal
        
            print(f"{filename}: {label} - Probabilidades: Normal: {prob_normal:.2f}, Abnormal: {prob_abnormal:.2f}")
'''